In [3]:
import pandas as pd

In [ ]:
df_pedidos = pd.read_csv('material/dados/olist_orders_dataset.csv')
df_pedidos

In [ ]:
df_pedidos.describe(include='all')

In [ ]:
df_itens_pedido = pd.read_csv('material/dados/olist_order_items_dataset.csv')
df_itens_pedido

In [ ]:
df_itens_pedido.describe(include='all')

In [ ]:
df_consumidor = pd.read_csv('dados/olist_customers_dataset.csv')
df_consumidor

In [ ]:
df_consumidor.describe(include='all')

In [ ]:
df_a = pd.DataFrame({'id_a': [1,2,3,4,8]})
df_a

In [ ]:
df_b = pd.DataFrame({'id_b': [1,2,3,4,5,6]})
df_b

In [ ]:
df_uniao = pd.merge(df_a, df_b, left_on='id_a', right_on='id_b', how='inner')
df_uniao

In [ ]:
df_esquerda = pd.merge(df_a, df_b, left_on='id_a', right_on='id_b', how='left')
df_esquerda

In [ ]:
df_direita = pd.merge(df_a, df_b, left_on='id_a', right_on='id_b', how='right')
df_direita

In [ ]:
df_tudo = pd.merge(df_a, df_b, left_on='id_a', right_on='id_b', how='outer')
df_tudo

In [ ]:
df_olist_pedidos = pd.merge(df_pedidos, df_itens_pedido, on=['order_id'], how='inner')
df_olist_pedidos

In [ ]:
df_olist_pedidos_uf = pd.merge(df_olist_pedidos, df_consumidor, on=['customer_id'], how='inner')
df_olist_pedidos_uf

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_pedidos.info()

In [ ]:
df_pedidos[df_pedidos.order_approved_at.isna()]

In [ ]:
columns_drop = ['order_approved_at', 'order_delivered_carrier_date', 'order_estimated_delivery_date']
df_pedidos = df_pedidos.drop(columns_drop, axis=1)
df_pedidos

In [ ]:
df_pedidos['order_status'].unique()

In [ ]:
df_pedidos['order_status'].value_counts()

In [ ]:
df_pedidos = df_pedidos.loc[df_pedidos['order_status'] == 'delivered']
df_pedidos

In [ ]:
# Filtrando somente pedidos finalizados, percebemos que perdemos um número razoável de linhas, cerca de 3 mil.
# Agora, com nosso dataset filtrado, precisamos resetar o seu index:
df_pedidos = df_pedidos.reset_index(drop=True)
df_pedidos

In [ ]:
df_pedidos.info()

In [ ]:
# Vamos transformar nossas colunas de data (que estão como string) em data: 
df_pedidos['order_purchase_timestamp'] = pd.to_datetime(df_pedidos['order_purchase_timestamp'])
df_pedidos['order_delivered_customer_date'] = pd.to_datetime(df_pedidos['order_delivered_customer_date'])

# Agora sim, vamos criar uma coluna de data e de mês!
df_pedidos['reference_date'] = df_pedidos['order_purchase_timestamp'].dt.date
df_pedidos['reference_month'] = df_pedidos['order_purchase_timestamp'].dt.strftime('%Y-%m-01')
df_pedidos['reference_month'] = pd.to_datetime(df_pedidos['reference_month'])
df_pedidos

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_itens_pedido.info()

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_consumidor.info()

In [ ]:
df_consumidor.head()

In [ ]:
df_consumidor = pd.read_csv('dados/olist_customers_dataset.csv', dtype={"customer_zip_code_prefix": "string"})
df_consumidor

In [ ]:
df_consumidor = df_consumidor.drop('customer_unique_id', axis=1)
df_consumidor

In [ ]:
# Jutando os df pedidos e itens de pedidos
df_olist = pd.merge(df_pedidos, df_itens_pedido, on='order_id', how='inner')
df_olist

In [ ]:
# Jutando agora com os dados de consumidor
df_olist = pd.merge(df_olist, df_consumidor, on='customer_id', how='inner')
df_olist


In [ ]:
# Exportando a base final
df_olist.to_csv('base_final_s14_olist.csv', index=False)

In [ ]:
# Aqui estamos buscando uma estatística descritiva, podemos utilizar a nossa função describe() para trazer essas informações
df_olist.describe().T

In [ ]:
df_olist['price'].mean()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(df_olist["price"], binwidth=50);

In [ ]:
df_sazonalidade = df_olist.groupby('reference_month').agg({'order_id': 'nunique'})
df_sazonalidade.sort_values(by='order_id', ascending=False)

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(data=df_sazonalidade.reset_index().sort_values("reference_month"), x="reference_month", y="order_id")
plt.xticks(rotation=45)

In [ ]:
date_sazonality = df_olist.groupby('reference_date').agg({'order_id': 'nunique'}).reset_index().sort_values("reference_date")

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(data=date_sazonality, 
             x="reference_date", y="order_id")

# Limitar o número de ticks no eixo x
plt.xticks(date_sazonality['reference_date'][::10], rotation=45)  # Altere o valor de '5' para outro intervalo desejado
;

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(data=date_sazonality[(date_sazonality.reference_date >= pd.to_datetime("2018-01-01")) & (date_sazonality.reference_date <= pd.to_datetime("2018-03-01"))], 
             x="reference_date", y="order_id")
plt.xticks(rotation=45);

In [ ]:
df_olist["day_of_week"] = df_olist["reference_date"].apply(lambda x: pd.to_datetime(x).day_of_week)
df_olist["day_name"] = df_olist["reference_date"].dt.day_name()

In [ ]:
df_olist[(df_olist.reference_date >= "2018-01-01") & (df_olist.reference_date <= "2018-03-01")].sort_values("reference_date")[["reference_date", "day_of_week", "day_name"]].drop_duplicates()

In [ ]:
df_olist["month"] = df_olist["reference_month"].apply(lambda x: pd.to_datetime(x).month)

month_sazonality = df_olist.groupby("month").agg({"order_id": "nunique"})

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(data=month_sazonality, 
             x="month", y="order_id")

In [ ]:
df_olist["year"] = df_olist["reference_date"].dt.year
year_month_sazonality = df_olist.groupby(["year", "month"]).agg({"order_id": "nunique"})

plt.figure(figsize=(15,5))
sns.lineplot(data=year_month_sazonality, 
             x="month", y="order_id", hue="year")

In [ ]:
print(df_olist.reference_date.min())
print(df_olist.reference_date.max())

In [ ]:
df_olist['order_delivered_customer_date'] = pd.to_datetime(df_olist['order_delivered_customer_date'])
df_olist['order_delivered_date'] = df_olist['order_delivered_customer_date'].dt.date

df_olist['shipping_limit_dt'] = pd.to_datetime(df_olist['shipping_limit_date']).dt.date

# condicional 
df_olist.loc[df_olist['order_delivered_date'] > df_olist['shipping_limit_dt'], 'status_entrega'] = 'ATRASADO'
df_olist.loc[df_olist['order_delivered_date'] < df_olist['shipping_limit_dt'], 'status_entrega'] = 'ADIANTADO'
df_olist.loc[df_olist['order_delivered_date'] == df_olist['shipping_limit_dt'], 'status_entrega'] = 'DENTRO DO ESPERADO'
df_olist

In [ ]:
df_olist["status_entrega"].value_counts(normalize=True)*100